# Running Panda and Lioness using the Graphical Processing Unit (GPU)

### Author: 
Daniel Morgan*.

*Channing division of network medicine, Brigham's and Women hospital and Harvard Medical School, Boston, MA.

## Introduction
PANDA [1] estimates gene regulatory networks using information from Transcription Factor (TF) Protein-Protein Interaction (PPI), gene coexpression, and TF DNA binding motifs. At its core, PANDA computes similarites between the three sources of information to infer the weight of regulatory edges, using matrix mulitplication. To accelerate the network inference, Graphical Processing Units (GPUs) can be used to perform matrix multiplication in an efficient manner.  

## 1. Setup Environment: download, install & import latest packages

In [0]:
import os
import s3fs
import pandas as pd
import numpy as np
from psutil import *

In [0]:
!nvcc --version

### 1.1 Configure cupy for installed Cuda version
1.   use cuda version based on above release (*e.g.* release 10.1= cuda101)
2.   install cupy version 7.4.0 or above 



In [2]:
!pip install cupy-cuda101==7.4.0
import cupy as cp
# print(cp.__version__)
!pip freeze | grep cupy

     |████████████████████████████████| 361.9MB 43kB/s 
  Found existing installation: cupy-cuda101 6.5.0
    Uninstalling cupy-cuda101-6.5.0:
      Successfully uninstalled cupy-cuda101-6.5.0
cupy-cuda101==7.4.0


##1.2 GPU and CPU info

In [ ]:
!nvidia-smi

In [ ]:
!cat /proc/cpuinfo

### 1.3 Clone netZoo

In [0]:
!rm -rf netZooPy

In [0]:
!git clone --single-branch --branch devel https://github.com/netZoo/netZooPy.git
os.chdir('netZooPy')
!pip install -e .
os.chdir('..')

In [0]:
import netZooPy
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness

## 2. Load test data

First, we get the data from the [GRAND database](https://grand.networkmedicine.org/download/). This can be done directly through pandas.

In [0]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/cells/ppi/LCL_ppi.txt',sep='\t',header=None)
LCL_expression=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/expression/Hugo_exp1_lcl.txt',sep='\t',header=None)
LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/gpuPANDA/Hugo_motifCellLine_reduced.txt',sep='\t',header=None)
LCL_ppi.to_csv('LCL_ppi.txt',sep='\t',index=False)
LCL_expression.to_csv('Hugo_exp1_lcl.txt',sep='\t',index=False)
LCL_motif.to_csv('Hugo_motifCellLine_reduced.txt',sep='\t',index=False)

In [0]:
# os.getcwd()
expression_data='Hugo_exp1_lcl.txt'
motif_data='Hugo_motifCellLine_reduced.txt'
ppi_data='LCL_ppi.txt'


## 3. Run Panda with GPU & precision flags

The precision flags allow to compute the network in sinlgle (7 decimal digits) or double precision (15 decimal digits). Although double precision networks are more accurate, single precision can be useful to free memory on devices and to go faster. It is a tradeoff that depends on the application

In [0]:
panda_obj = Panda(expression_data, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=True, keep_expression_matrix = True,modeProcess = 'intersection')

In [0]:
panda_obj.save_panda_results('single_cpu_panda.txt')

## 4. Run LIONESS with GPU

LIONESS [2] calls PANDA to estimate regulatory networks for each sample. We can use the GPU acceleration capabilities to estimate sample-specific networks through setting the `computing` flag to `gpu`.

In [0]:
lioness_obj = Lioness(panda_obj,computing='gpu',start=1, end=5, save_dir='lioness_output', save_fmt='txt')

In [ ]:
lioness_obj.panda_network.shape

### References
[1] Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013).

[2] Kuijjer, Marieke Lydia, et al. "Estimating sample-specific regulatory networks." iScience 14 (2019): 226-240.